<a href="https://colab.research.google.com/github/I-scream-icecream/Finance-dashboard/blob/main/predictorv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install necessary libraries and setup localtunnel

In [ ]:
!pip install streamlit yfinance pandas requests numpy matplotlib pandas_datareader keras scikit-learn selenium

In [ ]:
!npm install -g localtunnel

In [ ]:
%cd /content

Dynamic Market Insights Tool

In [ ]:
%%writefile DMIT.py

import streamlit as st
import numpy as np
import streamlit as st
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential, load_model
from sklearn.preprocessing import MinMaxScaler
import requests
from io import StringIO
import os

url_link = 'https://finance.yahoo.com/quote/%5ENSEI/components'
r = requests.get(url_link,headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'})

# Wrap the HTML content in StringIO
html_content = StringIO(r.text)

# Read the HTML content using pandas.read_html
reader = pd.read_html(html_content)[0]

dropdown = reader.Symbol.to_list()

st.title('Dynamic Market Insights Tool v2')

ticker = st.selectbox('Pick your assets',
                          dropdown)

start = st.date_input('Start', value = pd.to_datetime('2010-01-01'))
end = st.date_input('End', value = pd.to_datetime('today'))

df = yf.download(ticker,start,end)
df = df.reset_index()

# Describing data
st.subheader(f'Data from {start} to {end}')
st.write(df.describe())

# Visualization

st.subheader(f'Closing vs Time chart for {ticker} with 50MA & 200MA')

ma50 = df['Adj Close'].rolling(50).mean()

ma200 = df['Adj Close'].rolling(200).mean()

fig1 = plt.figure(figsize = (12,6))
plt.plot(df['Date'], df['Adj Close'])
plt.plot(df['Date'], ma200, 'r', label='200-day MA')
plt.plot(df['Date'], ma50, 'y', label='50-day MA')
plt.xlabel('Date')
plt.ylabel('Adj Close')
plt.title('Close Price Over Time')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.legend()
st.pyplot(fig1)


# Button to trigger LSTM model building
if st.button('Build LSTM Model'):
    # LSTM model building code
    try:
        st.subheader(f'Creating LSTM model for {ticker}. Please wait...')
        data_training = pd.DataFrame(df['Adj Close'][0:int(len(df)*0.70)])
        data_testing = pd.DataFrame(df['Adj Close'][int(len(df)*0.70): int(len(df))])

        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler(feature_range=(0,1))

        data_training_array = scaler.fit_transform(data_training)

        model_path = '/content/keras_model_{}_{}_to_{}.h5'.format(ticker, start, end)

        if os.path.exists(model_path):
            st.write("Model already exists. Continuing...")
            # Continue with your code here
            model = load_model('keras_model_{}_{}_to_{}.h5'.format(ticker, start, end))
        else:
            # Create training sequences and labels
            x_train = []
            y_train = []

            for i in range(100, data_training_array.shape[0]):
                x_train.append(data_training_array[i - 100: i])
                y_train.append(data_training_array[i, 0])

            x_train, y_train = np.array(x_train), np.array(y_train)

            model = Sequential()
            model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(x_train.shape[1], 1)))
            model.add(Dropout(0.2))

            model.add(LSTM(units=60, activation='relu', return_sequences=True))
            model.add(Dropout(0.3))

            model.add(LSTM(units=80, activation='relu', return_sequences=True))
            model.add(Dropout(0.4))

            model.add(LSTM(units=120, activation='relu'))
            model.add(Dropout(0.5))

            model.add(Dense(units=1))

            model.compile(optimizer='adam', loss='mean_squared_error')
            model.fit(x_train, y_train, epochs=50)

            model.save(f'keras_model_{ticker}_{start}_to_{end}.h5')

        past_100_days = data_training.tail(100)
        final_df = past_100_days.append(data_testing, ignore_index=True)
        input_data = scaler.fit_transform(final_df)

        x_test = []
        y_test = []

        for i in range(100, input_data.shape[0]):
          x_test.append(input_data[i-100: i])
          y_test.append(input_data[i, 0])

        x_test, y_test = np.array(x_test), np.array(y_test)
        y_predicted = model.predict(x_test)

        reverser = scaler.scale_[0]
        scale_factor = 1/reverser
        y_predicted = y_predicted * scale_factor
        y_test = y_test * scale_factor

        # Display success message
        st.success("LSTM Model built successfully!")

        fig2 = plt.figure(figsize=(12,6))
        plt.plot(y_test, 'b', label = 'Original Price')
        plt.plot(y_predicted, 'r', label = 'Predicted Price')
        plt.xlabel('Time')
        plt.ylabel('Price')
        plt.legend()
        st.pyplot(fig2)


    except Exception as e:
        # Display error message
        st.error(f"Error occurred: {e}")

Forexfactory webscraper

In [ ]:
!mkdir -p /content/pages
%cd /content/pages

In [ ]:
%%writefile forexfactory.py

import base64
import os
import streamlit as st
import pandas as pd
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

def create_driver():
    user_agent_list = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 11.5; rv:90.0) Gecko/20100101 Firefox/90.0',
        'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
        'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'
    ]
    user_agent = random.choice(user_agent_list)

    browser_options = webdriver.ChromeOptions()
    browser_options.add_argument("--no-sandbox")
    browser_options.add_argument("--headless")
    browser_options.add_argument("start-maximized")
    browser_options.add_argument("window-size=1900,6000")
    browser_options.add_argument("disable-gpu")
    browser_options.add_argument("--disable-software-rasterizer")
    browser_options.add_argument("--disable-dev-shm-usage")
    browser_options.add_argument(f'user-agent={user_agent}')

    driver = webdriver.Chrome(options=browser_options)

    return driver

def parse_data(driver, url):
    driver.get(url)

    data_table = driver.find_element(By.CLASS_NAME, "calendar__table")
    value_list = []

    for row in data_table.find_elements(By.TAG_NAME, "tr"):
        row_data = list(filter(None, [td.text for td in row.find_elements(By.TAG_NAME, "td")]))
        if row_data:
            value_list.append(row_data)  # Append only if row_data has content

    # Corrected variable name for consistency
    result_list = []
    currencies = ['CAD', 'JPY', 'USD', 'GBP', 'EUR', 'NZD', 'AUD', 'CHF', 'CNY']
    for row in value_list:
        if len(row) == 1:
            result_list.append(row)
        elif row[1] in currencies:
            result_list.append([None] + row)  # Add None for missing Date
        elif row[0] in currencies:
            result_list.append([None, None] + row)  # Add None for both Date and Time
        else:
            result_list.append(row)  # Keep the row as is

    # Create DataFrame with appropriate column names
    df = pd.DataFrame(result_list, columns=['Date', 'Time', 'Currency', 'Impact', 'Actual', 'Forecast', 'Previous'])

    # Save DataFrame to CSV
    df.to_csv("table_data.csv", index=False)
    st.write("CSV file generated successfully!")

def get_binary_file_downloader_html(bin_file, file_label='File'):
    with open(bin_file, 'rb') as f:
        data = f.read()
    b64_data = base64.b64encode(data).decode()
    href = f'<a href="data:file/csv;base64,{b64_data}" download="{bin_file}">{file_label}</a>'
    return href

# Streamlit UI
st.title('ForexFactory Web Scraper')

if st.button('Scrape Data'):
    with st.spinner('Scraping data...'):
        driver = create_driver()
        url = 'https://www.forexfactory.com/calendar'
        parse_data(driver=driver, url=url)

        # Display CSV file and provide download link
        if os.path.exists("/content/table_data.csv"):
            st.subheader("CSV file generated:")
            df = pd.read_csv("table_data.csv")
            st.write(df)

            st.markdown(get_binary_file_downloader_html("table_data.csv", 'CSV file'), unsafe_allow_html=True)




In [ ]:
%cd /content

Deploy

In [ ]:
!wget -q -O - ipv4.icanhazip.com

In [ ]:
!streamlit run DMIT.py & npx localtunnel --port 8501